In [1]:
import numpy as np
import tensorflow as tf
import sys
import time
from datetime import timedelta
import tensorflow.contrib.keras as kr
from sklearn import metrics
from sklearn.model_selection import KFold

import moxing as mox
mox.file.shift('os', 'mox')

INFO:root:Using MoXing-v1.14.1-ddfd6c9a
INFO:root:Using OBS-Python-SDK-3.1.2


In [2]:
trainDataPath = "s3://corpus-2/dataset/corpus_5_new.txt"
vocabPath = "s3://corpus-text-classification1/data/glove.6B.100d.txt"

In [3]:
split_info = {
    "random": False,
    "expert": [20, 4],
    "bundle": [920, 1],
    "table": [37, 3]
}


def dataset_split(info):
    if info:
        [num, pi] = info
        train_data = [[] for i in range(num)]
        with open(trainDataPath, "r", encoding='utf-8') as fp:
            for line in fp.readlines():
                word = line.split()
                info = word[0].split(":")
                index = int(info[pi]) - 1
                label = int(info[0])
                content = word[1:]
                train_data[index].append([content,label])

        for i in range(num):
            np.random.shuffle(train_data[i])
            train_data[i] = np.asarray(train_data[i])

        np.random.shuffle(train_data)   
        return train_data
    
    
    train_data = []
    with open(trainDataPath, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            word = line.split()
            label = int(word[0].split(":")[0])
            content = word[1:]
            train_data.append([content,label])
    
    np.random.shuffle(train_data)
    return np.asarray(train_data)


def mergeData(data_x, data_y):
    merge_x = data_x[0]
    merge_y = data_y[0]
    for i in range(1,len(data_x)):
        merge_x = np.r_[merge_x,data_x[i]]
        merge_y = np.r_[merge_y,data_y[i]]
        
    return merge_x, merge_y


def train_split_data(model, train_data, split_type):
    
    print(split_type)
    
    test_acc = []
    fold_id = 0
    
    if split_type != "random":
        tx = []
        ty = []
        for ti in train_data:
            x_train, y_train = process_file(ti[:,0], ti[:,1], word_to_id, num_classes, seq_length)
            tx.append(x_train)
            ty.append(y_train)

        tx = np.asarray(tx)
        ty = np.asarray(ty)

        print(len(tx),len(tx[0]),len(tx[1]),len(tx[0][0]))
        
        for train_i, test_i in kf.split(tx):
            fold_id += 1
            print("Fold: ", fold_id)
            train_x, train_y = mergeData(tx[train_i],ty[train_i])
            test_x, test_y = mergeData(tx[test_i],ty[test_i])
            test_acc.append(model_train(model, train_x, train_y, test_x, test_y, categories))
        
    else:
        tx, ty = process_file(train_data[:,0], train_data[:,1], word_to_id, num_classes, seq_length)
        print(len(tx),len(tx[0]),len(tx[1]))

        for train_i, test_i in kf.split(tx):
            fold_id += 1
            print("Fold: ", fold_id)
            test_acc.append(model_train(model,tx[train_i], ty[train_i],tx[test_i], ty[test_i],categories))
        
    print(test_acc)
    print("%s, %s, %s, %s" % (np.mean(test_acc),np.std(test_acc),np.std(test_acc,ddof=1),np.var(test_acc)))
    return test_acc

In [4]:
def loadGloVe(filename):
    vocab = []
    embd = []
    print('Loading GloVe!')
    # vocab.append('unk') #装载不认识的词
    # embd.append([0] * emb_size) #这个emb_size可能需要指定
    file = open(filename,'r',encoding='utf-8')
    for line in file.readlines():
        row = line.strip().split(' ')
        vocab.append(row[0])
        embd.append([float(ei) for ei in row[1:]])
    file.close()
    print('Completed!')
    return vocab,embd


def process_file(contents, labels, word_to_id, num_classes, pad_max_length):
    """
    将文件转换为id表示,并且将每个单独的样本长度固定为pad_max_lengtn
    """
    # contents, labels = readfile(filePath)
    data_id, label_id = [], []
    # 将文本内容转换为对应的id形式
    for i in range(len(contents)):
        data_id.append([word_to_id[x] for x in contents[i] if x in word_to_id])
        label_id.append(labels[i] - 1)  # label_id.append(cat_to_id[labels[i]])
    # 使用keras提供的pad_sequences来将文本pad为固定长度
    x_pad = kr.preprocessing.sequence.pad_sequences(data_id, pad_max_length)
    ''' https://blog.csdn.net/TH_NUM/article/details/80904900
    pad_sequences(sequences, maxlen=None, dtype=’int32’, padding=’pre’, truncating=’pre’, value=0.) 
        sequences：浮点数或整数构成的两层嵌套列表
        maxlen：None或整数，为序列的最大长度。大于此长度的序列将被截短，小于此长度的序列将在后部填0.
        dtype：返回的numpy array的数据类型
        padding：‘pre’或‘post’，确定当需要补0时，在序列的起始还是结尾补
        truncating：‘pre’或‘post’，确定当需要截断序列时，从起始还是结尾截断
        value：浮点数，此值将在填充时代替默认的填充值0
    '''
    y_pad = kr.utils.to_categorical(label_id, num_classes=num_classes)  # 将标签转换为one-hot表示
    ''' https://blog.csdn.net/nima1994/article/details/82468965
    to_categorical(y, num_classes=None, dtype='float32')
        将整型标签转为onehot。y为int数组，num_classes为标签类别总数，大于max(y)（标签从0开始的）。
        返回：如果num_classes=None，返回len(y) * [max(y)+1]（维度，m*n表示m行n列矩阵，下同），否则为len(y) * num_classes。
    '''
    return x_pad, y_pad

In [5]:
categories = ['Retrieve Value', 'Filter', 'Compute Derived Value', 'Find Extremum', 'Sort', 
                  'Determine Range', 'Characterize Distribution', 'Find Anomalies', 'Cluster', 'Correlate']
num_classes = len(categories)

vocab, embd = loadGloVe(vocabPath)
vocab_size = len(vocab)
embedding_dim = len(embd[0])
embedding = np.asarray(embd)
word_to_id = dict(zip(vocab, range(vocab_size)))

print(len(embedding),embedding_dim,vocab_size)
 
seq_length = 41  # seq_length = 37  TREC

Loading GloVe!
Completed!
400000 100 400000


In [6]:
def batch_iter(x_pad, y_pad, batch_size):
    """生成批次数据"""
    data_len = len(x_pad)
    num_batch = int((data_len - 1) / batch_size) + 1
    # np.arange()生成0到data_len的等差数列，默认等差为1；np.random.permutation()打乱生成的等差序列的顺序
    # 下面三句语句是为了将训练或测试文本的顺序打乱，因为原文本中每个分类的样本全部挨在一起，这样每个batch训练的都是同一个分类，不太好，打乱后每个batch可包含不同分类
    indices = np.random.permutation(np.arange(data_len))
    x_shuffle = x_pad[indices]
    y_shuffle = y_pad[indices]

    # 返回所有batch的数据
    for i in range(num_batch):
        start_id = i * batch_size
        end_id = min((i + 1) * batch_size, data_len)
        yield x_shuffle[start_id:end_id], y_shuffle[start_id:end_id]
        
        
def evaluate(sess, model, x_pad, y_pad, loss1, acc1, batch_size):
    """评估在某一数据上的准确率和损失"""
    data_len = len(x_pad)
    batch_eval = batch_iter(x_pad, y_pad, batch_size)  # 128
    total_loss = 0.0
    total_acc = 0.0
    for x_batch1, y_batch1 in batch_eval:
        batch_len = len(x_batch1)
        feed_dict1 = {model.inputX: x_batch1, model.inputY: y_batch1, model.dropoutKeepProb: 1.0}
        lossTmp, accTmp = sess.run([loss1, acc1], feed_dict=feed_dict1)
        total_loss += lossTmp * batch_len
        total_acc += accTmp * batch_len

    return total_loss / data_len, total_acc / data_len


def model_train(model, x_train, y_train, x_val, y_val, categories):
    
    # save_path = "%s/%s/%s/%s" % (savePath, split_type, fold_id, fold_id)
    # 创建session
    session = tf.Session()
    session.run(tf.global_variables_initializer())

    print('Training and evaluating...')
    
    total_batch = 0  # 总批次
    best_acc_train = 0.0  # 最佳验证集准确率
    last_improved = 0  # 记录上一次提升批次
    require_improvement = 500  # 如果超过1000轮未提升，提前结束训练
    flag = False

    for epoch in range(num_epochs):  # 20
        start_time = time.time()
        
        print('Epoch:', epoch + 1)
        batch_train = batch_iter(x_train, y_train, batch_size)
        for x_batch, y_batch in batch_train:
            feed_dict = {model.inputX: x_batch, model.inputY: y_batch, model.dropoutKeepProb: dropout_keep_prob}
            session.run(model.trainOp, feed_dict=feed_dict)  # 运行优化
            total_batch += 1

            if total_batch % print_per_batch == 0:
                # 每多少轮次输出在训练集和验证集上的性能
                feed_dict[model.dropoutKeepProb] = 1.0
                loss_train, acc_train = session.run([model.loss, model.acc], feed_dict=feed_dict)
                loss_val, acc_val = evaluate(session, model, x_val, y_val, model.loss, model.acc, 64)
                if acc_val > best_acc_train:
                    # 保存最好结果
                    best_acc_train = acc_val
                    last_improved = total_batch
                    # saver.save(sess=session, save_path=save_path)
                    improved_str = '*'
                else:
                    improved_str = ''
                
                duration = time.time() - start_time
                output = 'Iter: {:>1}, Train Loss: {:>6.4}, Train Acc: {:>6.2%}, Val Loss: {:>6.4}, Val Acc: {:>6.2%}, Time: {:.2f}s {}'
                print(output.format(total_batch, loss_train, acc_train, loss_val, acc_val, duration, improved_str))

            if total_batch - last_improved > require_improvement:
                # 验证集正确率长期不提升，提前结束训练
                print("No optimization for a long time, auto-stopping...")
                
                test_data_len = len(x_val)
                test_num_batch = int((test_data_len - 1) / batch_size) + 1

                y_test_cls = np.argmax(y_val, 1)  # 获得类别
                y_test_pred_cls = np.zeros(shape=len(x_val), dtype=np.int32)  # 保存预测结果  len(x_test) 表示有多少个文本

                for i in range(test_num_batch):  # 逐批次处理
                    start_id = i * batch_size
                    end_id = min((i + 1) * batch_size, test_data_len)
                    feed_dict = {
                        model.inputX: x_val[start_id:end_id],
                        model.dropoutKeepProb: 1.0
                    }
                    y_test_pred_cls[start_id:end_id] = session.run(model.y_pred_cls, feed_dict=feed_dict)

                accuracy_score = metrics.accuracy_score(y_test_cls, y_test_pred_cls)
                # 评估
                print("Precision, Recall and F1-Score...")
                print(metrics.classification_report(y_test_cls, y_test_pred_cls, target_names=categories))
                '''
                sklearn中的classification_report函数用于显示主要分类指标的文本报告．在报告中显示每个类的精确度，召回率，F1值等信息。
                    y_true：1维数组，或标签指示器数组/稀疏矩阵，目标值。 
                    y_pred：1维数组，或标签指示器数组/稀疏矩阵，分类器返回的估计值。 
                    labels：array，shape = [n_labels]，报表中包含的标签索引的可选列表。 
                    target_names：字符串列表，与标签匹配的可选显示名称（相同顺序）。 
                    原文链接：https://blog.csdn.net/akadiao/article/details/78788864
                '''

                # 混淆矩阵
                print("Confusion Matrix...")
                cm = metrics.confusion_matrix(y_test_cls, y_test_pred_cls)
                '''
                混淆矩阵是机器学习中总结分类模型预测结果的情形分析表，以矩阵形式将数据集中的记录按照真实的类别与分类模型作出的分类判断两个标准进行汇总。
                这个名字来源于它可以非常容易的表明多个类别是否有混淆（也就是一个class被预测成另一个class）
                https://blog.csdn.net/u011734144/article/details/80277225
                '''
                print(cm)
                
                flag = True
                break  # 跳出循环
        if flag:  # 同上
            break

    session.close()
    return accuracy_score

In [7]:
# 构建adversarailLSTM模型
class AdversarailLSTM(object):

    def __init__(self, wordEmbedding):
        # 定义输入
        self.inputX = tf.placeholder(tf.int32, [None, seq_length], name='inputX')
        self.inputY = tf.placeholder(tf.int32, [None, num_classes], name='inputY')

        self.dropoutKeepProb = tf.placeholder(tf.float64, name='keep_prob')

        # 词嵌入层
        with tf.name_scope("wordEmbedding"):
            wordEmbedding = tf.Variable(initial_value=wordEmbedding)
            self.embeddedWords = tf.nn.embedding_lookup(wordEmbedding, self.inputX)

        # 计算softmax交叉熵损失
        with tf.name_scope("loss"):
            with tf.variable_scope("Bi-LSTM", reuse=None):
                self.predictions = self._Bi_LSTMAttention(self.embeddedWords)
                # self.y_pred_cls = tf.cast(tf.greater_equal(self.predictions, 0.5), tf.float32, name="binaryPreds")
                self.y_pred_cls = tf.argmax(tf.nn.softmax(self.predictions),1)  # 预测类别 tf.argmax：返回每一行或每一列的最大值 1为里面（每一行），0为外面（每一列）
                # losses = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.predictions, labels=self.inputY)
                losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.predictions, labels=self.inputY)
                loss = tf.reduce_mean(losses)

        
        with tf.name_scope("perturloss"):
            with tf.variable_scope("Bi-LSTM", reuse=True):
                perturWordEmbedding = self._addPerturbation(self.embeddedWords, loss)
                print("perturbSize:{}".format(perturWordEmbedding))
                perturPredictions = self._Bi_LSTMAttention(perturWordEmbedding)
                # perturLosses = tf.nn.sigmoid_cross_entropy_with_logits(logits=perturPredictions, labels=self.inputY)
                perturLosses = tf.nn.softmax_cross_entropy_with_logits(logits=perturPredictions, labels=self.inputY)
                perturLoss = tf.reduce_mean(perturLosses)

        self.loss = loss + perturLoss
        
        globalStep = tf.Variable(0, name="globalStep", trainable=False)
        # 定义优化函数，传入学习速率参数
        optimizer = tf.train.AdamOptimizer(learning_rate)
        # 计算梯度,得到梯度和变量
        gradsAndVars = optimizer.compute_gradients(self.loss)
        # 将梯度应用到变量下，生成训练器
        self.trainOp = optimizer.apply_gradients(gradsAndVars, global_step=globalStep)

        # 准确率
        correct_pred = tf.equal(tf.argmax(self.inputY, 1), self.y_pred_cls)
        self.acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
        # self.loss = loss
        
        
    def _Bi_LSTMAttention(self, embeddedWords):
        # 定义两层双向LSTM的模型结构
        with tf.name_scope("Bi-LSTM"):
            fwHiddenLayers = []
            bwHiddenLayers = []
            for idx, hiddenSize in enumerate(hiddenSizes):
                with tf.name_scope("Bi-LSTM" + str(idx)):
                    # 定义前向网络结构
                    lstmFwCell = tf.nn.rnn_cell.DropoutWrapper(
                        tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                        output_keep_prob=self.dropoutKeepProb)

                    # 定义反向网络结构
                    lstmBwCell = tf.nn.rnn_cell.DropoutWrapper(
                        tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                        output_keep_prob=self.dropoutKeepProb)

                fwHiddenLayers.append(lstmFwCell)
                bwHiddenLayers.append(lstmBwCell)

            # 实现多层的LSTM结构， state_is_tuple=True，则状态会以元祖的形式组合(h, c)，否则列向拼接
            fwMultiLstm = tf.nn.rnn_cell.MultiRNNCell(cells=fwHiddenLayers, state_is_tuple=True)
            bwMultiLstm = tf.nn.rnn_cell.MultiRNNCell(cells=bwHiddenLayers, state_is_tuple=True)
            # 采用动态rnn，可以动态地输入序列的长度，若没有输入，则取序列的全长
            # outputs是一个元组(output_fw, output_bw), 其中两个元素的维度都是[batch_size, max_time, hidden_size], fw和bw的hiddensize一样
            # self.current_state是最终的状态，二元组(state_fw, state_bw), state_fw=[batch_size, s], s是一个元组(h, c)
            outputs, self.current_state = tf.nn.bidirectional_dynamic_rnn(fwMultiLstm, bwMultiLstm,
                                                                          self.embeddedWords, dtype=tf.float64,
                                                                          scope="bi-lstm" + str(idx))

        # 在bi-lstm+attention论文中，将前向和后向的输出相加
        with tf.name_scope("Attention"):
            H = outputs[0] + outputs[1]

            # 得到attention的输出
            output = self.attention(H)
            outputSize = hiddenSizes[-1]
            print("outputSize:{}".format(outputSize))

        # 全连接层的输出
        with tf.name_scope("output"):
            outputW = tf.get_variable(
                "outputW", dtype=tf.float64,
                shape=[outputSize, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())

            outputB = tf.Variable(tf.constant(0.1, dtype=tf.float64, shape=[num_classes]), name="outputB")

            predictions = tf.nn.xw_plus_b(output, outputW, outputB, name="predictions")

            return predictions

    def attention(self, H):
        """
        利用Attention机制得到句子的向量表示
        """
        # 获得最后一层lstm神经元的数量
        hiddenSize = hiddenSizes[-1]

        # 初始化一个权重向量，是可训练的参数
        W = tf.Variable(tf.random_normal([hiddenSize], stddev=0.1, dtype=tf.float64))

        # 对bi-lstm的输出用激活函数做非线性转换
        M = tf.tanh(H)

        # 对W和M做矩阵运算，W=[batch_size, time_step, hidden_size], 计算前做维度转换成[batch_size * time_step, hidden_size]
        # newM = [batch_size, time_step, 1], 每一个时间步的输出由向量转换成一个数字
        newM = tf.matmul(tf.reshape(M, [-1, hiddenSize]), tf.reshape(W, [-1, 1]))

        # 对newM做维度转换成[batch_size, time_step]
        restoreM = tf.reshape(newM, [-1, seq_length])

        # 用softmax做归一化处理[batch_size, time_step]
        self.alpha = tf.nn.softmax(restoreM)

        # 利用求得的alpha的值对H进行加权求和，用矩阵运算直接操作
        r = tf.matmul(tf.transpose(H, [0, 2, 1]), tf.reshape(self.alpha, [-1, seq_length, 1]))

        # 将三维压缩成二维sequeezeR = [batch_size, hissen_size]
        sequeezeR = tf.squeeze(r)

        sentenceRepren = tf.tanh(sequeezeR)

        # 对attention的输出可以做dropout处理
        output = tf.nn.dropout(sentenceRepren, self.dropoutKeepProb)

        return output

    def _normalize(self, wordEmbedding, weights):
        """
        对word embedding 结合权重做标准化处理
        """
        mean = tf.matmul(weights, wordEmbedding)
        powWordEmbedding = tf.pow(wordEmbedding - mean, 2.)

        var = tf.matmul(weights, powWordEmbedding)
        stddev = tf.sqrt(1e-6 + var)

        return (wordEmbedding - mean) / stddev

    def _addPerturbation(self, embedded, loss):
        """
        添加波动到word embedding
        """
        grad, = tf.gradients(
            loss,
            embedded,
            aggregation_method=tf.AggregationMethod.EXPERIMENTAL_ACCUMULATE_N)
        grad = tf.stop_gradient(grad)
        perturb = self._scaleL2(grad, epsilon)
        # print("perturbSize:{}".format(embedded+perturb))
        return embedded + perturb

    def _scaleL2(self, x, norm_length):
        # shape(x) = [batch, num_step, d]
        # divide x by max(abs(x)) for a numerically stable L2 norm
        # 2norm(x) = a * 2norm(x/a)
        # scale over the full sequence, dim(1, 2)
        alpha = tf.reduce_max(tf.abs(x), (1, 2), keep_dims=True) + 1e-12
        l2_norm = alpha * tf.sqrt(tf.reduce_sum(tf.pow(x / alpha, 2), (1, 2), keep_dims=True) + 1e-6)
        x_unit = x / l2_norm
        return norm_length * x_unit

In [8]:
hiddenSizes = [128]  # 定义LSTM的隐藏层（一层，128个神经元）
epsilon = 5

num_filters = 256
kernel_size = 5
hidden_dim = 128
learning_rate = 1e-3
dropout_keep_prob = 0.5

num_epochs = 50
batch_size = 64
print_per_batch = 30  # 每多少轮输出一次结果

lstm = AdversarailLSTM(embedding)

outputSize:128
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


perturbSize:Tensor("perturloss/Bi-LSTM/add_2:0", shape=(?, 41, 100), dtype=float64)
outputSize:128


In [10]:
split_info = {
    # "random": False,
    # "expert": [20, 4],
    "bundle": [920, 1],
    #"table": [37, 3]
}

kf = KFold(n_splits=10)
test_acc_split = []
for split_type,info in split_info.items():
    train_data = dataset_split(info)
    test_acc_split.append(train_split_data(lstm, train_data, split_type))

bundle
920 15 14 41
Fold:  1
Training and evaluating...
Epoch: 1
Iter: 30, Train Loss:   4.48, Train Acc: 17.19%, Val Loss:  4.512, Val Acc: 17.54%, Time: 7.11s *
Iter: 60, Train Loss:  3.564, Train Acc: 60.94%, Val Loss:  4.064, Val Acc: 31.68%, Time: 13.19s *
Iter: 90, Train Loss:  2.871, Train Acc: 65.62%, Val Loss:   3.42, Val Acc: 44.41%, Time: 19.22s *
Iter: 120, Train Loss:  2.333, Train Acc: 59.38%, Val Loss:   2.98, Val Acc: 50.71%, Time: 25.29s *
Iter: 150, Train Loss:  1.997, Train Acc: 70.31%, Val Loss:  2.652, Val Acc: 57.21%, Time: 31.28s *
Iter: 180, Train Loss:  1.551, Train Acc: 78.12%, Val Loss:  2.828, Val Acc: 54.74%, Time: 37.29s 
Epoch: 2
Iter: 210, Train Loss:  1.273, Train Acc: 81.25%, Val Loss:  2.668, Val Acc: 57.43%, Time: 3.33s *
Iter: 240, Train Loss:  1.426, Train Acc: 79.69%, Val Loss:  2.637, Val Acc: 61.74%, Time: 9.40s *
Iter: 270, Train Loss:  1.079, Train Acc: 84.38%, Val Loss:  2.466, Val Acc: 63.79%, Time: 15.39s *
Iter: 300, Train Loss: 0.4294, Tr

Iter: 1080, Train Loss: 0.1813, Train Acc: 96.88%, Val Loss:  2.454, Val Acc: 67.72%, Time: 18.24s 
Iter: 1110, Train Loss: 0.1873, Train Acc: 96.88%, Val Loss:  2.502, Val Acc: 66.60%, Time: 24.26s 
Iter: 1140, Train Loss: 0.09777, Train Acc: 100.00%, Val Loss:  2.417, Val Acc: 69.41%, Time: 30.21s 
Iter: 1170, Train Loss: 0.2302, Train Acc: 98.44%, Val Loss:   2.33, Val Acc: 69.13%, Time: 36.21s 
Epoch: 7
Iter: 1200, Train Loss: 0.01451, Train Acc: 100.00%, Val Loss:  2.652, Val Acc: 67.44%, Time: 3.33s 
Iter: 1230, Train Loss: 0.0223, Train Acc: 100.00%, Val Loss:  2.678, Val Acc: 68.00%, Time: 9.30s 
Iter: 1260, Train Loss: 0.3029, Train Acc: 95.31%, Val Loss:  2.578, Val Acc: 69.27%, Time: 15.34s 
Iter: 1290, Train Loss:  0.213, Train Acc: 96.88%, Val Loss:  2.747, Val Acc: 66.32%, Time: 21.32s 
Iter: 1320, Train Loss: 0.1789, Train Acc: 98.44%, Val Loss:   2.93, Val Acc: 66.24%, Time: 27.27s 
Iter: 1350, Train Loss: 0.03266, Train Acc: 100.00%, Val Loss:  2.794, Val Acc: 67.86%, 

Training and evaluating...
Epoch: 1
Iter: 30, Train Loss:  4.462, Train Acc: 12.50%, Val Loss:  4.665, Val Acc:  5.70%, Time: 7.01s *
Iter: 60, Train Loss:  4.027, Train Acc: 35.94%, Val Loss:  4.182, Val Acc: 29.49%, Time: 12.94s *
Iter: 90, Train Loss:  3.242, Train Acc: 50.00%, Val Loss:  3.287, Val Acc: 48.08%, Time: 18.87s *
Iter: 120, Train Loss:  2.604, Train Acc: 54.69%, Val Loss:  2.885, Val Acc: 55.13%, Time: 24.89s *
Iter: 150, Train Loss:  2.135, Train Acc: 67.19%, Val Loss:  2.595, Val Acc: 60.11%, Time: 30.92s *
Iter: 180, Train Loss:  1.505, Train Acc: 76.56%, Val Loss:  2.243, Val Acc: 65.81%, Time: 36.84s *
Epoch: 2
Iter: 210, Train Loss:   1.55, Train Acc: 73.44%, Val Loss:  2.025, Val Acc: 69.02%, Time: 3.27s *
Iter: 240, Train Loss:  0.968, Train Acc: 87.50%, Val Loss:  1.941, Val Acc: 69.66%, Time: 9.22s *
Iter: 270, Train Loss:    1.0, Train Acc: 89.06%, Val Loss:  1.853, Val Acc: 72.79%, Time: 15.14s *
Iter: 300, Train Loss: 0.7302, Train Acc: 92.19%, Val Loss:  

Iter: 990, Train Loss: 0.04438, Train Acc: 100.00%, Val Loss:  2.899, Val Acc: 67.07%, Time: 40.42s 
Epoch: 6
Iter: 1020, Train Loss: 0.1123, Train Acc: 100.00%, Val Loss:   3.08, Val Acc: 66.85%, Time: 6.01s 
Iter: 1050, Train Loss: 0.05018, Train Acc: 98.44%, Val Loss:  3.226, Val Acc: 67.28%, Time: 12.00s 
Iter: 1080, Train Loss: 0.1035, Train Acc: 100.00%, Val Loss:  3.172, Val Acc: 66.50%, Time: 18.01s 
Iter: 1110, Train Loss: 0.0466, Train Acc: 100.00%, Val Loss:  3.109, Val Acc: 67.56%, Time: 24.10s 
Iter: 1140, Train Loss: 0.05384, Train Acc: 100.00%, Val Loss:  2.995, Val Acc: 68.19%, Time: 30.16s 
Iter: 1170, Train Loss: 0.01673, Train Acc: 100.00%, Val Loss:  3.063, Val Acc: 69.32%, Time: 36.21s *
Epoch: 7
Iter: 1200, Train Loss: 0.2696, Train Acc: 95.31%, Val Loss:  3.205, Val Acc: 68.55%, Time: 3.32s 
Iter: 1230, Train Loss: 0.03057, Train Acc: 100.00%, Val Loss:  3.441, Val Acc: 67.77%, Time: 9.32s 
Iter: 1260, Train Loss: 0.05744, Train Acc: 100.00%, Val Loss:  3.382, Va

Iter: 1320, Train Loss: 0.0818, Train Acc: 98.44%, Val Loss:  2.738, Val Acc: 69.25%, Time: 27.03s 
Iter: 1350, Train Loss:   0.24, Train Acc: 96.88%, Val Loss:  2.668, Val Acc: 69.61%, Time: 33.03s 
Iter: 1380, Train Loss: 0.01129, Train Acc: 100.00%, Val Loss:  2.863, Val Acc: 69.25%, Time: 38.93s 
Epoch: 8
Iter: 1410, Train Loss: 0.008572, Train Acc: 100.00%, Val Loss:   3.01, Val Acc: 70.53%, Time: 5.08s 
Iter: 1440, Train Loss: 0.02655, Train Acc: 100.00%, Val Loss:  3.004, Val Acc: 69.96%, Time: 11.04s 
Iter: 1470, Train Loss: 0.01636, Train Acc: 100.00%, Val Loss:  3.044, Val Acc: 69.68%, Time: 16.90s 
Iter: 1500, Train Loss: 0.1023, Train Acc: 98.44%, Val Loss:  2.931, Val Acc: 70.25%, Time: 22.83s 
Iter: 1530, Train Loss: 0.03058, Train Acc: 100.00%, Val Loss:  2.933, Val Acc: 69.40%, Time: 28.75s 
Iter: 1560, Train Loss: 0.03356, Train Acc: 100.00%, Val Loss:  2.804, Val Acc: 70.75%, Time: 34.74s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Scor

Training and evaluating...
Epoch: 1
Iter: 30, Train Loss:  4.529, Train Acc: 10.94%, Val Loss:  4.501, Val Acc: 17.12%, Time: 7.01s *
Iter: 60, Train Loss:  3.966, Train Acc: 26.56%, Val Loss:  4.005, Val Acc: 34.97%, Time: 12.99s *
Iter: 90, Train Loss:  2.969, Train Acc: 46.88%, Val Loss:  3.734, Val Acc: 42.62%, Time: 18.90s *
Iter: 120, Train Loss:  2.478, Train Acc: 64.06%, Val Loss:  3.056, Val Acc: 56.21%, Time: 24.92s *
Iter: 150, Train Loss:  1.916, Train Acc: 67.19%, Val Loss:   2.76, Val Acc: 59.15%, Time: 30.77s *
Iter: 180, Train Loss:  1.825, Train Acc: 71.88%, Val Loss:   2.48, Val Acc: 63.34%, Time: 36.71s *
Epoch: 2
Iter: 210, Train Loss:  1.931, Train Acc: 68.75%, Val Loss:  2.574, Val Acc: 59.74%, Time: 3.12s 
Iter: 240, Train Loss:  1.338, Train Acc: 81.25%, Val Loss:  2.541, Val Acc: 60.54%, Time: 9.01s 
Iter: 270, Train Loss:   1.36, Train Acc: 76.56%, Val Loss:  2.822, Val Acc: 58.27%, Time: 14.89s 
Iter: 300, Train Loss: 0.9426, Train Acc: 87.50%, Val Loss:   2.

Iter: 660, Train Loss: 0.5952, Train Acc: 90.62%, Val Loss:  2.535, Val Acc: 69.94%, Time: 14.41s 
Iter: 690, Train Loss: 0.2174, Train Acc: 98.44%, Val Loss:  2.487, Val Acc: 67.96%, Time: 20.39s 
Iter: 720, Train Loss:  0.197, Train Acc: 98.44%, Val Loss:  2.611, Val Acc: 69.16%, Time: 26.40s 
Iter: 750, Train Loss: 0.1927, Train Acc: 98.44%, Val Loss:  2.577, Val Acc: 69.65%, Time: 32.36s 
Iter: 780, Train Loss: 0.3417, Train Acc: 96.88%, Val Loss:  2.387, Val Acc: 71.91%, Time: 38.30s *
Epoch: 5
Iter: 810, Train Loss:  0.238, Train Acc: 96.88%, Val Loss:  2.278, Val Acc: 73.47%, Time: 4.22s *
Iter: 840, Train Loss: 0.1826, Train Acc: 98.44%, Val Loss:  2.315, Val Acc: 71.98%, Time: 10.23s 
Iter: 870, Train Loss:  0.133, Train Acc: 98.44%, Val Loss:  2.235, Val Acc: 73.54%, Time: 16.24s *
Iter: 900, Train Loss: 0.1227, Train Acc: 96.88%, Val Loss:  2.589, Val Acc: 70.08%, Time: 22.24s 
Iter: 930, Train Loss: 0.09693, Train Acc: 100.00%, Val Loss:  2.698, Val Acc: 66.55%, Time: 28.30

In [9]:
split_info = {
    # "random": False,
    # "expert": [20, 4],
    #"bundle": [920, 1],
    "table": [37, 3]
}

kf = KFold(n_splits=10)
test_acc_split = []
for split_type,info in split_info.items():
    train_data = dataset_split(info)
    test_acc_split.append(train_split_data(lstm, train_data, split_type))

table
37 91 315 41
Fold:  1
Training and evaluating...
Epoch: 1
Iter: 30, Train Loss:  4.346, Train Acc: 21.88%, Val Loss:  4.517, Val Acc: 16.52%, Time: 8.67s *
Iter: 60, Train Loss:  3.729, Train Acc: 42.19%, Val Loss:  4.021, Val Acc: 32.59%, Time: 14.56s *
Iter: 90, Train Loss:  2.826, Train Acc: 53.12%, Val Loss:  3.537, Val Acc: 43.65%, Time: 20.61s *
Iter: 120, Train Loss:   2.32, Train Acc: 59.38%, Val Loss:  2.981, Val Acc: 48.51%, Time: 26.59s *
Iter: 150, Train Loss:  2.123, Train Acc: 62.50%, Val Loss:  2.859, Val Acc: 54.33%, Time: 32.62s *
Iter: 180, Train Loss:  1.685, Train Acc: 73.44%, Val Loss:  2.602, Val Acc: 56.28%, Time: 38.57s *
Epoch: 2
Iter: 210, Train Loss:  1.126, Train Acc: 89.06%, Val Loss:  2.623, Val Acc: 57.25%, Time: 3.06s *
Iter: 240, Train Loss:  1.011, Train Acc: 81.25%, Val Loss:  2.684, Val Acc: 55.23%, Time: 8.98s 
Iter: 270, Train Loss: 0.9333, Train Acc: 85.94%, Val Loss:  2.522, Val Acc: 59.57%, Time: 14.96s *
Iter: 300, Train Loss: 0.8401, Tra

Iter: 240, Train Loss:   1.12, Train Acc: 81.25%, Val Loss:  2.716, Val Acc: 55.41%, Time: 9.41s 
Iter: 270, Train Loss:  1.204, Train Acc: 85.94%, Val Loss:  2.859, Val Acc: 55.13%, Time: 15.53s 
Iter: 300, Train Loss: 0.8986, Train Acc: 87.50%, Val Loss:  2.597, Val Acc: 56.53%, Time: 21.71s *
Iter: 330, Train Loss:   1.07, Train Acc: 81.25%, Val Loss:  2.424, Val Acc: 59.83%, Time: 27.82s *
Iter: 360, Train Loss: 0.9322, Train Acc: 84.38%, Val Loss:  2.662, Val Acc: 57.09%, Time: 33.87s 
Iter: 390, Train Loss:   0.66, Train Acc: 93.75%, Val Loss:  2.664, Val Acc: 61.31%, Time: 40.02s *
Epoch: 3
Iter: 420, Train Loss: 0.5858, Train Acc: 89.06%, Val Loss:  2.737, Val Acc: 57.72%, Time: 5.17s 
Iter: 450, Train Loss:  0.517, Train Acc: 90.62%, Val Loss:  2.806, Val Acc: 56.18%, Time: 11.25s 
Iter: 480, Train Loss: 0.6053, Train Acc: 93.75%, Val Loss:  2.637, Val Acc: 59.06%, Time: 17.39s 
Iter: 510, Train Loss:  0.327, Train Acc: 93.75%, Val Loss:  2.818, Val Acc: 58.29%, Time: 23.57s 


Iter: 1380, Train Loss: 0.04614, Train Acc: 100.00%, Val Loss:  3.479, Val Acc: 63.95%, Time: 35.89s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.79      0.27      0.41       279
                   Filter       0.47      0.47      0.47       243
    Compute Derived Value       0.67      0.51      0.58       301
            Find Extremum       0.56      0.81      0.66       261
                     Sort       0.75      0.84      0.79       265
          Determine Range       0.58      0.67      0.62       319
Characterize Distribution       0.78      0.65      0.71       267
           Find Anomalies       0.70      0.57      0.63       270
                  Cluster       0.65      0.82      0.72       271
                Correlate       0.65      0.81      0.72       284

                micro avg       0.64      0.64      0.64      2760
        

Training and evaluating...
Epoch: 1
Iter: 30, Train Loss:  4.431, Train Acc: 17.19%, Val Loss:  4.489, Val Acc: 21.56%, Time: 6.89s *
Iter: 60, Train Loss:  4.063, Train Acc: 29.69%, Val Loss:  3.997, Val Acc: 34.87%, Time: 12.76s *
Iter: 90, Train Loss:  2.928, Train Acc: 50.00%, Val Loss:  3.232, Val Acc: 48.71%, Time: 18.52s *
Iter: 120, Train Loss:  2.007, Train Acc: 71.88%, Val Loss:  2.832, Val Acc: 52.62%, Time: 24.29s *
Iter: 150, Train Loss:  1.936, Train Acc: 68.75%, Val Loss:  2.598, Val Acc: 56.26%, Time: 30.08s *
Iter: 180, Train Loss:   1.61, Train Acc: 78.12%, Val Loss:  2.427, Val Acc: 62.29%, Time: 35.92s *
Epoch: 2
Iter: 210, Train Loss:  1.328, Train Acc: 82.81%, Val Loss:  2.321, Val Acc: 63.71%, Time: 2.45s *
Iter: 240, Train Loss: 0.7415, Train Acc: 87.50%, Val Loss:  2.534, Val Acc: 61.22%, Time: 8.29s 
Iter: 270, Train Loss:  1.189, Train Acc: 79.69%, Val Loss:  2.559, Val Acc: 62.47%, Time: 14.02s 
Iter: 300, Train Loss: 0.8755, Train Acc: 90.62%, Val Loss:   2

Iter: 720, Train Loss: 0.3506, Train Acc: 96.88%, Val Loss:  2.277, Val Acc: 71.31%, Time: 20.22s 
Iter: 750, Train Loss: 0.3449, Train Acc: 93.75%, Val Loss:  2.405, Val Acc: 70.79%, Time: 25.83s 
Iter: 780, Train Loss:   0.37, Train Acc: 93.75%, Val Loss:  2.278, Val Acc: 72.66%, Time: 31.36s 
Iter: 810, Train Loss: 0.1222, Train Acc: 100.00%, Val Loss:  2.333, Val Acc: 72.87%, Time: 36.92s 
Epoch: 5
Iter: 840, Train Loss: 0.1625, Train Acc: 96.88%, Val Loss:  2.283, Val Acc: 70.58%, Time: 4.09s 
Iter: 870, Train Loss: 0.1092, Train Acc: 100.00%, Val Loss:  2.684, Val Acc: 70.27%, Time: 9.67s 
Iter: 900, Train Loss:  0.203, Train Acc: 96.88%, Val Loss:  2.407, Val Acc: 73.91%, Time: 15.26s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.73      0.81      0.77        85
                   Filter       0.66      0.73      0.69       107
    Compute 

Iter: 450, Train Loss: 0.3499, Train Acc: 96.88%, Val Loss:  2.641, Val Acc: 64.02%, Time: 8.70s 
Iter: 480, Train Loss:  1.148, Train Acc: 87.50%, Val Loss:  2.692, Val Acc: 61.22%, Time: 14.41s 
Iter: 510, Train Loss: 0.4885, Train Acc: 95.31%, Val Loss:    2.5, Val Acc: 63.54%, Time: 20.00s 
Iter: 540, Train Loss:  0.332, Train Acc: 95.31%, Val Loss:  2.787, Val Acc: 60.64%, Time: 25.67s 
Iter: 570, Train Loss: 0.2563, Train Acc: 98.44%, Val Loss:  2.564, Val Acc: 63.54%, Time: 31.29s 
Iter: 600, Train Loss: 0.3503, Train Acc: 95.31%, Val Loss:  2.637, Val Acc: 63.93%, Time: 36.90s 
Epoch: 4
Iter: 630, Train Loss: 0.5856, Train Acc: 93.75%, Val Loss:  2.704, Val Acc: 66.92%, Time: 3.87s *
Iter: 660, Train Loss: 0.2968, Train Acc: 95.31%, Val Loss:  2.727, Val Acc: 63.93%, Time: 9.51s 
Iter: 690, Train Loss: 0.3432, Train Acc: 95.31%, Val Loss:  2.996, Val Acc: 62.28%, Time: 15.13s 
Iter: 720, Train Loss: 0.2841, Train Acc: 98.44%, Val Loss:  2.654, Val Acc: 63.44%, Time: 20.70s 
Ite